In [ ]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Dense, Activation, Input, Embedding, Reshape, MaxPooling1D, Conv1D
from keras.layers import LSTM, GRU, Conv1D
from keras.layers import Dropout, BatchNormalization, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.activations import sigmoid
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.callbacks import TensorBoard
from nltk.tokenize import wordpunct_tokenize
import pymorphy2 as pm
import numpy as np
import random
import sys
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
with open('data/merged_sampled.json', encoding='utf-8') as f:
    json_data = json.load(f)

In [ ]:
original_sentences = [wordpunct_tokenize(x) for x in (list(json_data.keys()))]
fake_sentences = []
for x in json_data.values():
    fake_sentences.extend([wordpunct_tokenize(_) for _ in x])
    
print('original_sentences:\t', len(original_sentences))
print('fake_sentences:\t', len(fake_sentences))

print('mean len of original sentences:\t', np.mean([len(x) for x in original_sentences]), 'words')
print(' max len of original sentences:\t', np.max([len(x) for x in original_sentences]), 'words')
print('mean len of fake sentences:\t', np.mean([len(x) for x in fake_sentences]), 'words')
print(' max len of fake sentences:\t', np.max([len(x) for x in fake_sentences]), 'words')

In [ ]:
a = plt.hist([len(x) for x in original_sentences], bins=30)

In [ ]:
a = plt.hist([len(x) for x in fake_sentences], bins=30)

In [ ]:
max_len = 40

In [ ]:
%%time
morph = pm.MorphAnalyzer()

tag2id = dict()
id2tag = dict()
word2tag = dict()
tags_count = 0
for sent in original_sentences + fake_sentences:
    for word in sent:
        if not (word in word2tag.keys()):
            tag = morph.parse(word)[0].tag
            word2tag[word] = tag
            if not (tag in tag2id.keys()):
                tag2id[tag] = tags_count
                id2tag[tags_count] = tag
                tags_count += 1
print(len(tag2id))
print(len(word2tag))

In [ ]:
from libs.utils import pad

# transform text into sequence of indices
pad_idx = tags_count
original_indexes = np.array([pad([tag2id[word2tag[word]] for word in sent], max_len, pad_idx) for sent in original_sentences])
fake_indexes     = np.array([pad([tag2id[word2tag[word]] for word in sent], max_len, pad_idx) for sent in fake_sentences])

In [ ]:
n_batches = (len(original_indexes) + len(fake_indexes)) // 300
batch_size = 16

In [ ]:
from libs.utils import split_data_into_correct_batches
X, y = split_data_into_correct_batches(original_indexes, fake_indexes, n_batches, max_len, make_equal_folding=True)

In [ ]:
X.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

a = plt.hist(y)

In [ ]:
from keras.losses import sparse_categorical_crossentropy

def create_cnn():
    inp = Input(shape=(max_len,), dtype="int32")
    v = Embedding(tags_count+1, int(tags_count / 4))(inp)
    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(v)
    x = Dropout(0.3)(BatchNormalization()(x))
    x = MaxPooling1D(4, padding='same')(x)

    x = Conv1D(256, kernel_size=4, activation='relu', padding='same')(x)
    x = Dropout(0.3)(BatchNormalization()(x))
    x = MaxPooling1D(5, padding='same')(x)

    h = Flatten()(x) # None, 5*256
    y = Dense(2, activation='softmax')(h)
    model = Model(inp, y, name="char_cnn")
    model_to_save = Model(inp, y, 'char_cnn')
    model.compile(optimizer=RMSprop(), loss=sparse_categorical_crossentropy, metrics=['accuracy'])
    return model, model_to_save

In [ ]:
nn, nn_to_save = create_cnn()

In [ ]:
print(nn.summary())

In [ ]:
from time import clock

n_epochs = 20
n_batches = (len(original_indexes) + len(fake_indexes)) // 15

lens = [max_len - np.mean([list(x).count(tags_count) for x in fake_indexes])]
sizes = [len(fake_indexes)]

indexes = None
for epoch in range(n_epochs):
    if epoch == 0:
        X, y = split_data_into_correct_batches(original_indexes, fake_indexes, n_batches, max_len, make_equal_folding=True)
    elif epoch % 3 == 0:
        t = clock()
        probs = nn.predict(fake_indexes)[:, 0]
        bool_ind = np.random.uniform(0., 1., probs.shape) < probs
        indexes = np.arange(bool_ind.shape[0])[bool_ind]

        print('epoch', epoch, '- deleting took', clock() - t, 'sec')
        X, y = split_data_into_correct_batches(original_indexes, fake_indexes[indexes], n_batches, max_len, make_equal_folding=True)
        lens.append(max_len - np.mean([list(x).count(tags_count) for x in fake_indexes[indexes]]))
        sizes.append(len(indexes))
    nn.fit(X, y, batch_size=batch_size, shuffle=True, validation_split=0.2)

In [ ]:
a = plt.hist([len(x) for x in original_sentences], bins=30)

In [ ]:
a = plt.hist([len(x) for x in fake_sentences[indexes]], bins=30)

In [ ]:
lens

In [ ]:
nn_to_save.save('models/discriminator_believability_rnn_model.h5')